Combine three comments file into an integrated comments file

In [ ]:
import pandas as pd

# Step 1: List your three comment files manually
files = [
    r"C:\Users\User\Desktop\Datathon\Dataset-20250902T161158Z-1-001\Dataset\dataset\comments1.csv",
    r"C:\Users\User\Desktop\Datathon\Dataset-20250902T161158Z-1-001\Dataset\dataset\comments2.csv",
    r"C:\Users\User\Desktop\Datathon\Dataset-20250902T161158Z-1-001\Dataset\dataset\comments3.csv"
]

# Step 2: Read and combine
df_list = [pd.read_csv(f) for f in files]
combined = pd.concat(df_list, ignore_index=True)

# Step 3: Remove duplicates (optional, based on commentId)
if "commentId" in combined.columns:
    total_before = combined.shape[0]
    combined = combined.drop_duplicates(subset=["commentId"])
    total_after = combined.shape[0]
    duplicates_removed = total_before - total_after
else:
    duplicates_removed = 0
    total_after = combined.shape[0]

# Step 4: Save as integrated file
output_path = r"C:\Users\User\Desktop\Datathon\comments_integrated.csv"
combined.to_csv(output_path, index=False, encoding="utf-8")


print("✅ Integrated file saved at:", output_path)
print("Total rows after cleaning:", total_after)
print("Duplicates removed:", duplicates_removed)


✅ Integrated file saved at: C:\Users\User\Desktop\Datathon\comments_integrated.csv
Total rows after cleaning: 2999998
Duplicates removed: 0


Merge comments and videos file using inner join (videoid)

In [ ]:
import pandas as pd
import numpy as np

# ====== 1) Replace these with your local file paths ======
comments_path = r"C:\Users\User\Desktop\Datathon\comments_integrated.csv"
videos_path   = r"C:\Users\User\Desktop\Datathon\Dataset-20250902T161158Z-1-001\Dataset\dataset\videos.csv"
output_path   = r"C:\Users\User\Desktop\Datathon\comments_videos_integrated.csv"

# ====== 2) Load files ======
comments = pd.read_csv(comments_path)
videos   = pd.read_csv(videos_path)

# Defensive: ensure videoId exists
if "videoId" not in comments.columns or "videoId" not in videos.columns:
    raise ValueError("Both files must contain a 'videoId' column.")

# ====== 3) Rename columns as requested ======
# Comments side
# - likeCount -> comment_likeCount
# - publishedAt -> comment_publishedAt
# - 'kind' -> replaced by new 'comment' column with sequential labels per video
if "likeCount" in comments.columns:
    comments = comments.rename(columns={"likeCount": "comment_likeCount"})
if "publishedAt" in comments.columns:
    comments = comments.rename(columns={"publishedAt": "comment_publishedAt"})

# Create sequential 'comment' labels per video (comment 1, comment 2, ...)
# If you prefer one global sequence, replace groupby with a simple range.
comments = comments.copy()
comments["_seq"] = comments.groupby("videoId").cumcount() + 1
comments["comment"] = "comment " + comments["_seq"].astype(str)
# If an original 'kind' column exists, drop it; we’ve replaced it with 'comment'
if "kind" in comments.columns:
    comments = comments.drop(columns=["kind"])
comments = comments.drop(columns=["_seq"])

# ====== 4) Clean/drop unwanted columns on videos side ======
# You asked to remove:
#   "kind", "channelID", "defaultlangusge", "defaultaudiolanguage", "favourite count"
# Actual columns in your videos.csv appear as:
#   kind, channelId, defaultLanguage, defaultAudioLanguage, favouriteCount
# We'll drop using a tolerant matcher.
to_drop_variants = {
    "kind": ["kind"],
    "channelId": ["channelId", "channelID", "ChannelId", "ChannelID"],
    "defaultLanguage": ["defaultLanguage", "defaultlangusge", "defaultLangusge"],
    "defaultAudioLanguage": ["defaultAudioLanguage", "defaultaudiolanguage"],
    "favouriteCount": ["favouriteCount", "favourite count", "favoriteCount", "favorite count"]
}
drop_cols = []
for canonical, variants in to_drop_variants.items():
    for v in variants:
        if v in videos.columns:
            drop_cols.append(v)
# Drop duplicates in the drop list, then drop from df
drop_cols = list(dict.fromkeys(drop_cols))
videos = videos.drop(columns=[c for c in drop_cols if c in videos.columns], errors="ignore")

# ====== 5) Rename videos columns as requested ======
# - likeCount -> video_likeCount
# - publishedAt -> video_publishedAt
if "likeCount" in videos.columns:
    videos = videos.rename(columns={"likeCount": "video_likeCount"})
if "publishedAt" in videos.columns:
    videos = videos.rename(columns={"publishedAt": "video_publishedAt"})

# ====== 6) Count eliminations BEFORE inner join ======
comments_rows_before = comments.shape[0]
videos_rows_before   = videos.shape[0]

# Rows eliminated due to NO matching videoId (pre-compute per-file eliminations)
vid_ids_in_videos   = set(videos["videoId"].astype(str))
vid_ids_in_comments = set(comments["videoId"].astype(str))

comments_eliminated = comments[~comments["videoId"].astype(str).isin(vid_ids_in_videos)].shape[0]
videos_eliminated   = videos[~videos["videoId"].astype(str).isin(vid_ids_in_comments)].shape[0]

# ====== 7) Inner join on videoId ======
merged = pd.merge(
    comments,
    videos,
    on="videoId",
    how="inner",
    suffixes=("", "_video")  # we already renamed key overlaps
)

# ====== 8) Save + report ======
merged.to_csv(output_path, index=False, encoding="utf-8")

print("✅ Merge complete.")
print(f"Output saved to: {output_path}")
print(f"Final merged shape: {merged.shape[0]} rows × {merged.shape[1]} cols")

print("\n🧮 Rows eliminated due to no matching videoId (prior to inner join):")
print(f"- From comments: {comments_eliminated} row(s) (out of {comments_rows_before})")
print(f"- From videos:   {videos_eliminated} row(s) (out of {videos_rows_before})")

# Optional: quick peek at columns
print("\nColumns in merged:")
print(list(merged.columns))


✅ Merge complete.
Output saved to: C:\Users\User\Desktop\Datathon\comments_videos_integrated.csv
Final merged shape: 1048231 rows × 19 cols

🧮 Rows eliminated due to no matching videoId (prior to inner join):
- From comments: 344 row(s) (out of 1048575)
- From videos:   68081 row(s) (out of 92759)

Columns in merged:
['commentId', 'channelId', 'videoId', 'authorId', 'textOriginal', 'parentCommentId', 'comment_likeCount', 'comment_publishedAt', 'updatedAt', 'comment', 'video_publishedAt', 'title', 'description', 'tags', 'contentDuration', 'viewCount', 'video_likeCount', 'commentCount', 'topicCategories']


Further Detail Cleaning

In [ ]:
import pandas as pd
import re
import unicodedata
import sys
!pip install ftfy emoji

# =========================
# 1) REQUIRED DEPENDENCIES
# =========================
# ftfy: robust Unicode repair (fixes mojibake like "Ã¢ÂÂ¤" -> "❤")
try:
    from ftfy import fix_text
except Exception:
    raise ImportError("ftfy is required. Install with: pip install ftfy")

# emoji: convert ALL emojis (incl. flags/ZWJ) to readable English
try:
    import emoji
except Exception:
    raise ImportError("emoji is required. Install with: pip install emoji")

# =========================
# 2) PATHS (EDIT THESE)
# =========================
input_path  = r"C:\Users\User\Desktop\Datathon\comments_videos_integrated.csv"
output_path = r"C:\Users\User\Desktop\Datathon\comments_videos_cleaned.csv"

# =========================
# 3) HELPERS
# =========================
URL_SAFE_CHARS = set("@:/?=.&%+-_")  # keep URL/handle punctuation so links/mentions survive

# Heuristic markers that often appear in mojibake
_MOJI_MARKERS_RE = re.compile(r"[ÃÂâð]")

def robust_fix_mojibake(s: str) -> str:
    """
    Repair broken Unicode like 'Ã¢ÂÂ¤' -> '❤'
    Strategy:
      1) ftfy.fix_text()
      2) If tell-tale markers remain, try latin-1 -> utf-8
      3) If still present, try cp1252 -> utf-8
      4) ftfy again
    """
    if not isinstance(s, str):
        return s
    t = s
    try:
        t = fix_text(t)
    except Exception:
        pass
    if _MOJI_MARKERS_RE.search(t):
        try:
            t = t.encode("latin-1", errors="ignore").decode("utf-8", errors="ignore")
        except Exception:
            pass
    if _MOJI_MARKERS_RE.search(t):
        try:
            t = t.encode("cp1252", errors="ignore").decode("utf-8", errors="ignore")
        except Exception:
            pass
    try:
        t = fix_text(t)
    except Exception:
        pass
    return t

def emoji_to_words_all(s: str) -> str:
    """
    Convert every emoji to plain English words.
    - Uses emoji.replace_emoji when available (handles complex ZWJ/flags).
    - Falls back to demojize -> strip colons/underscores.
    """
    if not isinstance(s, str):
        return s

    # Prefer replace_emoji (emoji>=2.0) for clean names without colons
    try:
        # data may be dict; 'en' provides English CLDR name (e.g., 'red_heart')
        def _repl(ch, data):
            if isinstance(data, dict):
                name = data.get("en", "")
            else:
                name = str(data)
            # normalize name: underscores -> spaces, trim
            name = name.replace("_", " ").strip()
            return name
        s2 = emoji.replace_emoji(s, replace=_repl)
    except Exception:
        # Fallback: demojize -> :shortcode: then strip colons and underscores
        try:
            short = emoji.demojize(s, language="en")
            s2 = re.sub(r":([a-zA-Z0-9_]+):", lambda m: m.group(1).replace("_", " "), short)
        except Exception:
            s2 = s

    # Optional: collapse double spaces that can result from replacements
    s2 = re.sub(r"\s+", " ", s2).strip()
    return s2

def remove_unnecessary_punct_preserve_urls(text: str) -> str:
    """
    Remove punctuation EXCEPT URL/handle chars; keep letters/numbers/spaces.
    Run this AFTER emoji_to_words_all so any colons in names won't linger.
    """
    if not isinstance(text, str):
        return text
    out = []
    for ch in text:
        if unicodedata.category(ch).startswith("P"):
            if ch in URL_SAFE_CHARS:
                out.append(ch)   # keep URL/handle punctuation
            # else drop
        else:
            out.append(ch)
    return "".join(out)

def clean_comment_inplace(s: str) -> str:
    """
    FULL CLEAN for textOriginal (in place):
      - Mojibake repair -> real Unicode
      - Unicode normalize (NFKC)
      - Convert ALL emojis -> words (❤ -> 'red heart', 🇮🇳 -> 'flag India', etc.)
      - Lowercase
      - Remove '#' (keep hashtag words)
      - Replace newlines/tabs with spaces
      - Remove unnecessary punctuation (preserve URL/handle chars)
      - Collapse spaces
    """
    if not isinstance(s, str):
        return s
    s = robust_fix_mojibake(s)
    s = unicodedata.normalize("NFKC", s)
    s = emoji_to_words_all(s)   # **key step**: every emoji becomes text
    s = s.lower()
    s = s.replace("#", "")
    s = s.replace("\n", " ").replace("\r", " ").replace("\t", " ")
    s = remove_unnecessary_punct_preserve_urls(s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def clean_title_inplace(title: str) -> str:
    """
    CLEAN for title (in place, lighter):
      - Mojibake repair
      - Unicode normalize
      - Convert ALL emojis -> words (same as comments)
      - Keep case (no forced lowercase)
      - Minimal spacing tidy
    """
    if not isinstance(title, str):
        return title
    t = robust_fix_mojibake(title)
    t = unicodedata.normalize("NFKC", t)
    t = emoji_to_words_all(t)
    t = re.sub(r"\s+", " ", t).strip()
    return t

# =========================
# 4) “ONLY-NOISE” FILTERS
# =========================
MONTHS = r"(jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:t|tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)"
ONLY_INT_DECIMAL      = re.compile(r"^\d+(?:\.\d+)?$")
ONLY_PERCENT          = re.compile(r"^\d+(?:\.\d+)?\s*%$")
ONLY_TIME             = re.compile(r"^\d{1,2}:\d{1,2}(?::\d{1,2})?$")
ONLY_MONTH            = re.compile(rf"^{MONTHS}$", re.I)
MONTH_DAY_1           = re.compile(rf"^{MONTHS}\s+\d{{1,2}}$", re.I)
MONTH_DAY_2           = re.compile(rf"^\d{{1,2}}\s+{MONTHS}$", re.I)
ONLY_WHITESPACE_EMPTY = re.compile(r"^\s*$")

def is_only_noise(t: object) -> bool:
    """
    True if the (cleaned) string is only empty/number/percent/time/month/month+day.
    Non-strings are treated as noise.
    """
    if not isinstance(t, str):
        return True
    s = t.strip()
    return (
        bool(ONLY_WHITESPACE_EMPTY.match(s))
        or bool(ONLY_INT_DECIMAL.match(s))
        or bool(ONLY_PERCENT.match(s))
        or bool(ONLY_TIME.match(s))
        or bool(ONLY_MONTH.match(s))
        or bool(MONTH_DAY_1.match(s))
        or bool(MONTH_DAY_2.match(s))
    )

# =========================
# 5) MAIN
# =========================
def main():
    # Read CSV (pandas < 2.0 compatible)
    df = pd.read_csv(input_path, encoding="utf-8")

    # Ensure required column exists
    if "textOriginal" not in df.columns:
        raise ValueError("Column 'textOriginal' not found in the input file.")

    # Clean/convert title IN PLACE (if present)
    if "title" in df.columns:
        df["title"] = df["title"].apply(clean_title_inplace)
    else:
        print("⚠️  Column 'title' not found; skipping title processing.")

    # Clean/convert textOriginal IN PLACE
    df["textOriginal"] = df["textOriginal"].apply(clean_comment_inplace)

    # Remove only-noise rows (based on cleaned textOriginal)
    rows_before = len(df)
    mask_noise = df["textOriginal"].apply(is_only_noise)
    removed_count = int(mask_noise.sum())
    df = df.loc[~mask_noise].reset_index(drop=True)

    # Save
    df.to_csv(output_path, index=False, encoding="utf-8")

    # Report
    print("✅ Cleaning complete (mojibake fixed, all emojis converted to words).")
    print(f"→ Saved to: {output_path}")
    print(f"Rows before: {rows_before:,}")
    print(f"Removed (only-number/percent/time/date-like/empty): {removed_count:,}")
    print(f"Rows after:  {len(df):,}")

    # Quick preview
    preview_cols = ["textOriginal"]
    if "title" in df.columns:
        preview_cols.append("title")
    print("\nPreview:")
    try:
        print(df[preview_cols].head(10).to_string(index=False))
    except Exception:
        print(df.head(10).to_string(index=False))

if __name__ == "__main__":
    main()


   ---------------------------------------- 0.0/590.6 kB ? eta -:--:--
   --------------------------------------- 590.6/590.6 kB 10.3 MB/s eta 0:00:00
✅ Cleaning complete (mojibake fixed, all emojis converted to words).
→ Saved to: C:\Users\User\Desktop\Datathon\comments_videos_cleaned.csv
Rows before: 1,048,231
Removed (only-number/percent/time/date-like/empty): 4,396
Rows after:  1,043,835

Preview:
                                                                                                                                                                                                                                                                                       textOriginal                                                                                                                            title
                                                                                                                                                                              

In [ ]:
# === One-shot cleaner: CLEAN (not drop) punctuation in textOriginal + title; then drop empty/non-meaningful + duplicates ===
import pandas as pd
import re
from pathlib import Path

# ==== CONFIG ====
INPUT_CSV  = r"C:\Users\User\Desktop\Datathon\comments_videos_cleaned.csv"    # change if needed
OUTPUT_CSV = r"C:\Users\User\Desktop\Datathon\comments_videos_cleaned(2).csv"

# ==== 1) Load with robust encoding ====
def robust_read_csv(path):
    try:
        return pd.read_csv(path, encoding="utf-8")
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding="latin1")

df = robust_read_csv(INPUT_CSV)
orig_rows = len(df)

# Ensure target columns exist
cols_to_clean = [c for c in ["textOriginal", "title"] if c in df.columns]
if not cols_to_clean:
    raise ValueError("Neither 'textOriginal' nor 'title' column found in the CSV.")

# ==== 2) Define a compact English stopword set (no downloads required) ====
STOPWORDS = {
    "a","about","above","after","again","against","all","am","an","and","any","are","as","at",
    "be","because","been","before","being","below","between","both","but","by",
    "can","could",
    "did","do","does","doing","down","during",
    "each","few","for","from","further",
    "had","has","have","having","he","her","here","hers","herself","him","himself","his","how",
    "i","if","in","into","is","it","its","itself",
    "just",
    "me","more","most","my","myself",
    "no","nor","not","now",
    "of","off","on","once","only","or","other","our","ours","ourselves","out","over","own",
    "same","she","should","so","some","such",
    "than","that","the","their","theirs","them","themselves","then","there","these","they",
    "this","those","through","to","too",
    "under","until","up",
    "very",
    "was","we","were","what","when","where","which","while","who","whom","why","with",
    "you","your","yours","yourself","yourselves",
    # Extras common in scraped text
    "im","ive","youre","youve","dont","doesnt","didnt","cant","wont","isnt","arent","wasnt","werent",
    "hey","hi","ok","okay","yeah","ya","oh","uh","uhm","um","hmm","lol","haha","hahaha","amp"
}

# ==== 3) Cleaning helpers ====
PUNCT_NUM_NONALPHA = re.compile(r"[^A-Za-z\s]+")  # keep letters and spaces only
MULTI_SPACE = re.compile(r"\s+")

def clean_text(s: str) -> str:
    if pd.isna(s):
        return ""
    # lower
    s = str(s).lower()
    # remove specific noise explicitly (covers '::', '...', unicode ellipsis, em-dashes, etc.)
    s = s.replace("::", " ").replace("...", " ").replace("…", " ")
    # keep only letters/spaces (remove punctuation, digits, emojis, symbols)
    s = PUNCT_NUM_NONALPHA.sub(" ", s)
    # collapse spaces
    s = MULTI_SPACE.sub(" ", s).strip()
    # stopword removal
    if not s:
        return ""
    tokens = [t for t in s.split() if t not in STOPWORDS]
    # rejoin
    cleaned = " ".join(tokens)
    # final collapse (in case)
    cleaned = MULTI_SPACE.sub(" ", cleaned).strip()
    return cleaned

def is_meaningful(s: str) -> bool:
    """Consider meaningful if at least one token of length >= 2 remains."""
    if not s:
        return False
    toks = s.split()
    return any(len(t) >= 2 for t in toks)

# ==== 4) Apply cleaning to the two columns (modify in place) ====
for c in cols_to_clean:
    df[c] = df[c].apply(clean_text)

# ==== 5) Drop rows where BOTH fields are empty or non-meaningful after cleaning ====
meaning_mask = pd.Series(False, index=df.index)
if all(col in df.columns for col in ["textOriginal","title"]):
    meaning_mask = df["textOriginal"].apply(is_meaningful) | df["title"].apply(is_meaningful)
else:
    # If only one exists, require it to be meaningful
    only_col = cols_to_clean[0]
    meaning_mask = df[only_col].apply(is_meaningful)

dropped_empty_or_nonmeaning = int((~meaning_mask).sum())
df = df.loc[meaning_mask].copy()

# ==== 6) Drop duplicates on the cleaned textOriginal + title ====
if all(c in df.columns for c in ["textOriginal","title"]):
    dup_count = int(df.duplicated(subset=["textOriginal","title"]).sum())
    df = df.drop_duplicates(subset=["textOriginal","title"], keep="first")
else:
    dup_count = int(df.duplicated(subset=cols_to_clean).sum())
    df = df.drop_duplicates(subset=cols_to_clean, keep="first")

# ==== 7) Save and report ====
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")

print("=== Cleaning Report ===")
print(f"Input file                     : {Path(INPUT_CSV).resolve()}")
print(f"Output file                    : {Path(OUTPUT_CSV).resolve()}")
print(f"Original rows                  : {orig_rows}")
print(f"Dropped empty/non-meaning rows : {dropped_empty_or_nonmeaning}")
print(f"Dropped duplicates             : {dup_count}")
print(f"Final rows                     : {len(df)}")
print("\nNotes:")
print("- Cleaned IN-ROW punctuation like '::' and '...' (did not drop rows just for having them).")
print("- Kept only alphabet characters, removed numbers/symbols/punctuation.")
print("- Removed common English stopwords to keep analyzable tokens.")
print("- Dropped rows that became empty/non-meaningful after cleaning, and removed duplicates.")


=== Cleaning Report ===
Input file                     : C:\Users\User\Desktop\Datathon\comments_videos_cleaned.csv
Output file                    : C:\Users\User\Desktop\Datathon\comments_videos_cleaned(2).csv
Original rows                  : 1043835
Dropped empty/non-meaning rows : 47
Dropped duplicates             : 154006
Final rows                     : 889782

Notes:
- Cleaned IN-ROW punctuation like '::' and '...' (did not drop rows just for having them).
- Kept only alphabet characters, removed numbers/symbols/punctuation.
- Removed common English stopwords to keep analyzable tokens.
- Dropped rows that became empty/non-meaningful after cleaning, and removed duplicates.
